## Importing all required libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
import joblib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from exception import AtisException
import os,sys

In [2]:
# Download the stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

#pd.read_csv("data/atis_intents_train.csv",header=None)

In [4]:
#pd.read_csv("data/atis_intents_test.csv",header=None)

In [5]:
columns = ['intent','text']

In [6]:
df = pd.read_csv("data/atis_intents.csv",header=None,names=columns)

In [7]:
df

,intent,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
...,...,...
4973,atis_airfare,what is the airfare for flights from denver t...
4974,atis_flight,do you have any flights from denver to baltim...
4975,atis_airline,which airlines fly into and out of denver
4976,atis_flight,does continental fly from boston to san franc...


In [8]:
df.loc[2]

intent                                     atis_flight_time
text       what is the arrival time in san francisco for...
Name: 2, dtype: object

In [9]:
df['text'].sample(15)


2051     display all flights from san francisco to bos...
2902     show me one way flights from milwaukee to orl...
1339     i need to fly from washington to san francisc...
164      what flights are there on delta from boston t...
4246               show me flights from new york to miami
3740     does eastern airlines fly between boston and ...
3785                   from phoenix to denver on a monday
1427                   ground transportation in baltimore
4839     list the shortest flight from san jose to hou...
200             list daily flights from boston to atlanta
943      are there any nonstop flights from indianapol...
3637     what flights from las vegas to montreal on sa...
563                      what does restriction ap 57 mean
1637     do you have a flight from salt lake city to s...
2904     in the month of december are there any flight...
Name: text, dtype: object

## Information about the data

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4978 entries, 0 to 4977
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   intent  4978 non-null   object
 1   text    4978 non-null   object
dtypes: object(2)
memory usage: 77.9+ KB


In [11]:
df.shape

(4978, 2)

In [12]:
df.describe()

,intent,text
count,4978,4978
unique,22,4634
top,atis_flight,what is fare code h
freq,3666,8


In [13]:
df['intent'].value_counts()

intent
atis_flight                                 3666
atis_airfare                                 423
atis_ground_service                          255
atis_airline                                 157
atis_abbreviation                            147
atis_aircraft                                 81
atis_flight_time                              54
atis_quantity                                 51
atis_flight#atis_airfare                      21
atis_airport                                  20
atis_distance                                 20
atis_city                                     19
atis_ground_fare                              18
atis_capacity                                 16
atis_flight_no                                12
atis_meal                                      6
atis_restriction                               6
atis_airline#atis_flight_no                    2
atis_ground_service#atis_ground_fare           1
atis_airfare#atis_flight_time                  1
atis_cheapest

##### Missing Values checking

In [14]:
df.isnull().sum()

intent    0
text      0
dtype: int64

In [15]:
df[df.duplicated()]

,intent,text
261,atis_abbreviation,what is fare code h
278,atis_flight,pittsburgh to denver
283,atis_flight,flights from boston to pittsburgh
285,atis_airfare,show me the fares from dallas to san francisco
542,atis_flight,show me flights from pittsburgh to philadelphia
...,...,...
4936,atis_flight,newark to cleveland
4939,atis_flight,show me flights from denver to philadelphia
4940,atis_ground_service,show me ground transportation in denver
4949,atis_flight,show me the flights from baltimore to oakland


In [16]:
(
    df
    .loc[df.duplicated(keep=False)]
    .sort_values(['text'])
)

,intent,text
3795,atis_airport,airports in new york
2412,atis_airport,airports in new york
2874,atis_flight#atis_airfare,all flights and fares from atlanta to dallas ...
602,atis_flight#atis_airfare,all flights and fares from atlanta to dallas ...
859,atis_flight#atis_airfare,all flights and fares from atlanta to dallas ...
...,...,...
64,atis_airline,which airlines have first class flights today
4329,atis_flight,which flights are between boston and baltimor...
1481,atis_flight,which flights are between boston and baltimor...
2012,atis_flight,which united airlines flights go through denver


#### Dropping the Duplicated data

In [17]:
df = df.drop_duplicates()

In [18]:
df.duplicated().sum()

0

## Data Preprocessing

In [19]:
# Define a function for text cleaning
def clean_text(text):
    try:
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word.isalnum()]
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        return ' '.join(tokens)
    except Exception as e:
        raise AtisException(e,sys)

df['text'] = df['text'].apply(clean_text)

# Encode labels
label_encoder = LabelEncoder()
df['intent'] = label_encoder.fit_transform(df['intent'])


label_encoder_filename = 'label_encoder.joblib'
joblib.dump(label_encoder, label_encoder_filename)

# Display the first few rows to verify the changes
df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14292\3118857762.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14292\3118857762.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['intent'] = label_encoder.fit_transform(df['intent'])


,intent,text
0,12,want fly boston 838 arrive denver 1110 morning
1,12,flights available pittsburgh baltimore thursda...
2,15,arrival time san francisco 755 flight leaving ...
3,3,cheapest airfare tacoma orlando
4,3,round trip fares pittsburgh philadelphia 1000 ...


In [20]:
pd.DataFrame(df['intent'].value_counts())

,count
intent,
12,3426
3,403
17,235
5,148
0,108
1,78
15,52
20,49
11,20


In [21]:

tfidf_vector = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 3))

X = tfidf_vector.fit_transform(df['text'])

print(f"TF-IDF matrix shape: {X.shape}")

# Save the TF-IDF vectorizer to a file
vectorizer_filename = 'tfidf_vectorizer_ngram.joblib'
joblib.dump(tfidf_vector, vectorizer_filename)
print(f"TF-IDF vectorizer saved to {vectorizer_filename}")


df.head()

TF-IDF matrix shape: (4634, 5000)
TF-IDF vectorizer saved to tfidf_vectorizer_ngram.joblib


,intent,text
0,12,want fly boston 838 arrive denver 1110 morning
1,12,flights available pittsburgh baltimore thursda...
2,15,arrival time san francisco 755 flight leaving ...
3,3,cheapest airfare tacoma orlando
4,3,round trip fares pittsburgh philadelphia 1000 ...


## Model Training 

In [22]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['intent'], test_size=0.2, random_state=42)

# Initialize the models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate the models
results = {}
for model_name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        results[model_name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        }
    except Exception as e:
        raise AtisException(e,sys)
#print the results
results


C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

{'Logistic Regression': {'Accuracy': 0.8813376483279396,
  'Precision': 0.8603983633080451,
  'Recall': 0.8813376483279396,
  'F1 Score': 0.8534389519874469},
 'SVM': {'Accuracy': 0.9277238403451996,
  'Precision': 0.9221302387845821,
  'Recall': 0.9277238403451996,
  'F1 Score': 0.9159594778355201},
 'Random Forest': {'Accuracy': 0.9223300970873787,
  'Precision': 0.9191844355562371,
  'Recall': 0.9223300970873787,
  'F1 Score': 0.9133288989694733}}

In [23]:
pd.DataFrame(results)

,Logistic Regression,SVM,Random Forest
Accuracy,0.881338,0.927724,0.922330
Precision,0.860398,0.922130,0.919184
Recall,0.881338,0.927724,0.922330
F1 Score,0.853439,0.915959,0.913329


In [24]:
max_words = 5000
max_len = 100

# Initialize the Tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])

# Convert the text to sequences and pad them
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Splitting the data into training and testing sets
X_train_seq, X_test_seq, y_train, y_test = train_test_split(padded_sequences, df['intent'], test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(22, activation='softmax'))  # 22 is the number of unique intents

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with 10 epochs and used early_stopping 
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train_seq, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred_seq = model.predict(X_test_seq)
y_pred_classes = y_pred_seq.argmax(axis=-1)

accuracy = accuracy_score(y_test, y_pred_classes)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='weighted')

# Display the evaluation metrics
lstm_results = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1
}

lstm_results

Epoch 1/10
47/47 [==============================] - 10s 165ms/step - loss: 1.5311 - accuracy: 0.7204 - val_loss: 1.0296 - val_accuracy: 0.7749
Epoch 2/10
47/47 [==============================] - 8s 163ms/step - loss: 1.1647 - accuracy: 0.7288 - val_loss: 1.0253 - val_accuracy: 0.7749
Epoch 3/10
47/47 [==============================] - 8s 168ms/step - loss: 1.1635 - accuracy: 0.7288 - val_loss: 1.0407 - val_accuracy: 0.7749
Epoch 4/10
47/47 [==============================] - 8s 163ms/step - loss: 1.1633 - accuracy: 0.7288 - val_loss: 1.0259 - val_accuracy: 0.7749
Epoch 5/10
29/29 [==============================] - 1s 13ms/step


C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Accuracy': 0.7443365695792881,
 'Precision': 0.5540369288130623,
 'Recall': 0.7443365695792881,
 'F1 Score': 0.6352408571548654}

## Observations

* The SVM model is the best among the four models including LSTM based on the provided metrics.
* LSTM did not work well in this sue less data Neural Network work better when w ehave huge data
* It has the highest accuracy, precision, recall, and F1 score, making it the most effective model for intent classification on this dataset.

## Hyper Parameter Tuning

In [25]:
# Define the parameter grid for Grid Search
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['linear', 'rbf']
}

# Initialize the SVM model
svm = SVC(random_state=42)

# Initialize Grid Search with Cross-Validation
grid_search = GridSearchCV(svm, param_grid, refit=True, verbose=2, cv=5, n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Score: {best_score}")

# Train the best model on the entire training set
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, y_train)

# Predict on the test set
y_pred_best = best_svm.predict(X_test)

# Calculate evaluation metrics
best_accuracy = accuracy_score(y_test, y_pred_best)
best_precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_best, average='weighted')

# Display the evaluation metrics
print(f"Accuracy: {best_accuracy}")
print(f"Precision: {best_precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Save the model to a file
model_filename = 'best_svm_model_ngram.joblib'
joblib.dump(best_svm, model_filename)

print(f"Model saved to {model_filename}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Parameters: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
Best Cross-Validation Score: 0.9425410405549431
Accuracy: 0.9320388349514563
Precision: 0.9311815542677491
Recall: 0.9320388349514563
F1 Score: 0.9290386721077591
Model saved to best_svm_model_ngram.joblib


C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Obervation after the Hyper parameter Tuning

* The SVM model with tuned hyperparameters is performing excellently, with high accuracy, precision, recall, and F1 score. 
* The slight imbalance in class predictions will be corrected by SMOTE but which isnt required almost SVM model works well without it